In [ ]:
import pandas as pd
import numpy as np
import nltk

In [ ]:
nltk.download('punkt')

In [ ]:
import os
import nltk.corpus
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize

In [ ]:
# read result 
result = pd.read_csv("result.csv") 
Tags = result["Tag"]
Tags

In [ ]:
allTag = []
for row in result.index: 
    list1 = list(map(str, result['Tag'][row].replace('[','').replace(']','').replace('\'','').split(',')))
    for x in list1: 
        allTag.append(x.replace(' ', ''))


In [ ]:
# find most popular 20tag 
# token = word_tokenize(allTag)
token = allTag
fdist = FreqDist(token)
fdist20 = fdist.most_common(20)
fdist20

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.style.use('ggplot')

x = []
count = []
for i in range(len(fdist20)): 
    x.append(fdist20[i][0])
    count.append(fdist20[i][1])

x_pos = [i for i, _ in enumerate(x)]
plt.barh(x_pos, count, color = '#f81c1c')
for index, value in enumerate(count):
    plt.text(value, index, str(value))

plt.xticks(rotation=90)
plt.ylabel("Tag")
plt.xlabel("Number of Listing")
plt.title("Most Common tags")

plt.yticks(x_pos, x)

plt.show()

In [ ]:
# clean up price
price = result["Price/Budget"]
price = price.str.replace('Avg Bid', '')

for i in range(len(price)): 
    price[i] = price[i].replace('$', '')
    # handel range price
    if '-' in price[i]: 
        a,b = price[i].split('-')
        if '/ hr' in a: 
            a = float(a.replace('/ hr', '')) 
            # * 40
        else: 
            a = float(a)
        if '/ hr' in b: 
            b = float(b.replace('/ hr', '')) 
            # * 40
        else: 
            b = float(b)
        price[i] = (a+b)/2
    else: 
        if 'min' in price[i] or '/ hr' in price[i]: 
            price[i] = price[i].replace('min', '')
            price[i] = float(price[i].replace('/ hr', '')) 
            # * 40
        price[i] = float(price[i])
totalPrice = sum(price)
averagePrice = price.mean()
averagePrice

In [ ]:
# get bid weighted average price 
Bids = result["Bids/Entries"]
Bids = Bids.str.replace(' bids', '').str.replace(' entries', '')
Bids = Bids.astype(int)
sumBids = sum(Bids)
bidWeight = Bids.map(lambda x : x/sumBids)
bidWeight

In [ ]:
# get tag frequency weight
total = 0
dictFdist = dict(fdist)
for x in dictFdist: 
    total += dictFdist[x]
for x in dictFdist: 
    dictFdist[x] = (dictFdist[x]/total)
freqWeight = dictFdist

In [ ]:
# get total weighted value of each tag 
labels = {}
for i in range(len(price)): 
    list1 = list(map(str, result['Tag'][i].replace('[','').replace(']','').replace('\'','').split(',')))
    for x in list1: 
        x = x.replace(' ','')
        # priceWeight = price[i] * bidWeight[i]
        if x not in labels:
            labels[x] = float(0)
        else: 
            labels[x] += (price[i] * freqWeight[x] * bidWeight[i])

In [ ]:
# tag value index 
plt.style.use('ggplot')

x = []
count = []
for i in range(len(fdist20)): 
    key = fdist20[i][0]
    x.append(key)
    count.append(labels[key])

x_pos = [i for i, _ in enumerate(x)]
plt.barh(x_pos, count, color = '#f81c1c')
for index, value in enumerate(count):
    plt.text(value, index, str("{:.6f}".format(value)))

plt.ylabel("Tag")
plt.xlabel("Tag average value")
plt.title("Tag value")

plt.yticks(x_pos, x)

plt.show()

In [ ]:
# sorted tag value index
labels = sorted(labels.items(), key=lambda x: x[1], reverse=True)

plt.style.use('ggplot')

x = []
count = []
for i in range(0, 20): 
    key = list(labels)[i][0]
    x.append(key)
    count.append(dict(labels)[key])

x_pos = [i for i, _ in enumerate(x)]
plt.barh(x_pos, count, color = '#f81c1c')
for index, value in enumerate(count):
    plt.text(value, index, str("{:.6f}".format(value)))

plt.ylabel("Tag")
plt.xlabel("Tag average value")
plt.title("Tag value (sorted)")

plt.yticks(x_pos, x)

plt.show()